In [1]:
import os
import json
import random
import numpy as np
import pandas as pd
import math
import torch
import gpytorch
import sys
import re
import statistics
from matplotlib import pyplot as plt
from scipy.stats import gaussian_kde
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

In [2]:
import simple_gp
from Kidera import kidera

In [3]:
df = pd.read_csv('data/DDG_Dataset.csv')
# df = pd.read_csv('data/DDG_Dataset_mini.csv')
df = df[df['Temperature'] != "'-"].reset_index(drop=True)
df['Temperature'] = df['Temperature'].astype('float64')
ndata = df.shape[0]
print(df.columns)
print(ndata)
df

Index(['Rowid', 'ID', 'Protein Name', 'Mutation', 'Chain', 'PDB',
       'Temperature', 'pH', 'ΔΔG', 'Reference', 'A', 'R', 'N', 'D', 'C', 'Q',
       'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V',
       'ASP', 'PHE', 'GLN', 'LYS', 'ILE', 'TYR', 'GLY', 'ASN', 'ARG', 'LEU',
       'TRP', 'ALA', 'THR', 'VAL', 'HIS', 'CYS', 'GLU', 'MET', 'PRO', 'SER'],
      dtype='object')
8535


,Rowid,ID,Protein Name,Mutation,Chain,PDB,Temperature,pH,ΔΔG,Reference,...,TRP,ALA,THR,VAL,HIS,CYS,GLU,MET,PRO,SER
0,1,1,Tryptophan synthase alpha chain,E49M,A,1WQ5,298.95,7.0,4.60,PMID: 378988,...,0.0,4.0,2.0,4.0,0.0,0.0,0.0,1.0,2.0,2.0
1,2,2,Tryptophan synthase alpha chain,E49Q,A,1WQ5,298.95,7.0,-2.50,PMID: 378988,...,0.0,4.0,2.0,4.0,0.0,0.0,0.0,1.0,2.0,2.0
2,3,3,Endolysin,W138Y,A,2LZM,298.15,2.2,-1.71,PMID: 911878,...,0.0,5.0,4.0,4.0,0.0,0.0,2.0,2.0,1.0,2.0
3,4,6,Cellular tumor antigen p53,C242S,A,1TUP,283.15,7.2,-3.07,PMID: 1203434,...,0.0,0.0,0.0,1.0,2.0,4.0,2.0,3.0,1.0,2.0
4,5,7,Cellular tumor antigen p53,F134L,A,1TUP,283.15,7.2,-4.78,PMID: 1203434,...,0.0,2.0,5.0,3.0,0.0,5.0,4.0,1.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8530,8570,13318,Transcriptional repressor arc,V25A,A,1ARR,298.15,7.5,-0.40,PMID: 7664079,...,0.0,1.0,0.0,3.0,0.0,0.0,4.0,0.0,0.0,1.0
8531,8571,13319,Transcriptional repressor arc,V18A,A,1ARR,298.15,7.5,-0.50,PMID: 7664079,...,1.0,1.0,0.0,3.0,0.0,0.0,2.0,0.0,1.0,0.0
8532,8572,13320,Transcriptional repressor arc,V33A,A,1ARR,298.15,7.5,-2.10,PMID: 7664079,...,1.0,1.0,0.0,4.0,0.0,0.0,3.0,0.0,0.0,2.0
8533,8573,13321,Transcriptional repressor arc,S32A,A,1ARR,298.15,7.5,-3.80,PMID: 7664079,...,0.0,1.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,1.0


In [4]:
# df['Temperature'] = df['Temperature'].astype('float64')
df['Temperature'] = df['Temperature'].astype('float64')
# print(df['Temperature'].unique())
data_x = df.drop(columns=['Rowid', 'ID', 'Protein Name', 'Mutation', 'Chain', 'PDB', 'ΔΔG', 'Reference']).to_numpy()
data_y = df['ΔΔG'].to_numpy()

In [ ]:
%%time

os.system('mkdir -p data/boruta_2')
for r in range(data_x.shape[1]-1, -1, -1):
    tmp_data_x = np.copy(data_x)
    random_list = random.choices(sorted(np.unique(tmp_data_x[:, r])), k=ndata)
    tmp_data_x[:, r] = random_list
    # pred_y, lower_y, upper_y = simple_gp.solve(tmp_data_x, data_y, 5, 5)
    pred_y, lower_y, upper_y = simple_gp.solve(tmp_data_x, data_y, 5, 100)
    tmp_df = df.copy()
    tmp_df['rand_items'] = random_list
    tmp_df['pred_ΔΔG'] = pred_y
    tmp_df['lower_ΔΔG'] = lower_y
    tmp_df['upper_ΔΔG'] = upper_y
    tmp_df.to_csv('data/boruta_2/pred_DDG_Dataset_gpytorch_Kidera_{}.csv'.format(r))

Iter 10/100 - Loss: 2.23659
Iter 20/100 - Loss: 2.07514
Iter 30/100 - Loss: 2.01110
Iter 40/100 - Loss: 1.98607
Iter 50/100 - Loss: 1.97717
Iter 60/100 - Loss: 1.97188
Iter 70/100 - Loss: 1.97308
Iter 80/100 - Loss: 1.97329
Iter 90/100 - Loss: 1.97254
Iter 100/100 - Loss: 1.96848
Iter 10/100 - Loss: 2.23132
Iter 20/100 - Loss: 2.07504
Iter 30/100 - Loss: 2.00890
Iter 40/100 - Loss: 1.98079
Iter 50/100 - Loss: 1.97903
Iter 60/100 - Loss: 1.97330
Iter 70/100 - Loss: 1.97225
Iter 80/100 - Loss: 1.97055
Iter 90/100 - Loss: 1.97147
Iter 100/100 - Loss: 1.96686
Iter 10/100 - Loss: 2.24285
Iter 20/100 - Loss: 2.08083
Iter 30/100 - Loss: 2.01761
Iter 40/100 - Loss: 1.99405
Iter 50/100 - Loss: 1.98512
Iter 60/100 - Loss: 1.98374
Iter 70/100 - Loss: 1.98034
Iter 80/100 - Loss: 1.98319
Iter 90/100 - Loss: 1.97737
Iter 100/100 - Loss: 1.97364
Iter 10/100 - Loss: 2.23105
Iter 20/100 - Loss: 2.07551
Iter 30/100 - Loss: 2.00966
Iter 40/100 - Loss: 1.97928
Iter 50/100 - Loss: 1.97321
Iter 60/100 - Los